In [48]:
from Bio import SeqIO
import numpy as np
import pandas as pd

/Users/seiya/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [54]:
FULLDATASET = 'dataset/ecamidata.fa'
PARTIALDATASET =  'dataset/testdata.fa'

/Users/seiya/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
with open(FULLDATASET) as fasta_file:  # Will close handle cleanly
    identifiers = []
    sequences = []
    effects = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifier = seq_record.id.split('|')[0]
        effect = seq_record.id.split('|')[1:]
        identifiers.append(identifier)
        sequences.append(seq_record.seq.__str__())
        effects.append(effect)

/Users/seiya/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
# identifiers = np.array(identifiers)
# sequences = np.array(sequences)
# effects = np.array(effects)

/Users/seiya/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
s1 = pd.Series(identifiers, name='ID')
s2 = pd.Series(sequences, name='sequence')
s3 = pd.Series(effects, name='effect')
Qfasta = pd.DataFrame(dict(ID=s1, sequence=s2, effect=s3)).set_index(['ID'])

/Users/seiya/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
Qfasta

/Users/seiya/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sequence,effect
ID,,
AWI06117.1,MIVQSTNSNRIGENLLKLGYISEEQLEIALKIQKKTNKLIGNVLVE...,[GT2]
AWI06118.1,MLSVVVPVYNEEKNVEELVKRIKAVLTNIEYEIVFVDDSVDNTPQI...,[GT2]
AWH92887.1,MSQSPGWARFAGRELPTWYDDVKLGVFVHWGPYSVPRWAPRVPDIQ...,[GH29]
AWH89110.1,MGELYPVLKVSVVIPVYNEEESLPALLSRTIAACKQLKQPYEIILV...,[GT2]
AWH84670.1,MKKLIALACFLLAVVNAANAQELKSPNGAFTMKFSLQAGGVPTYSL...,[GH97]
...,...,...
EEC72531.1,MAELRHSTVAAAAARSSNSPAKRDSDASAASSPFASTSSARGRGGG...,[GH106]
EEC81207.1,MAELRHATAAAAATRASSSPAKRDAESYASSSPFVASSSSPRGGGG...,[GH106]
EEE52851.1,MSVALYARGGGCGAKAAAARRSGRGGGGRRRAAVVLLLALAYVAGL...,[GH106]
